In [5]:
from torch_geometric.nn.models.lightgcn import LightGCN
import pandas as pd
import os
from tqdm import tqdm
import torch

c:\Users\Varun\anaconda3\envs\p38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data
We can begin by loading in the user review data. For each user, we have a subset of the movies that they reviewed. We'll load each of the CSVs as dataframes, and store a dict of user IDs corresponding to their dataframes.

In [6]:
# for now we will use the first 10k rows of the data, set to None to use all data
AMOUNT_TO_LOAD = 20000
EMBEDDING_DIM = 128

In [7]:
user_reviews_dir = 'user_reviews'
user_review_data = dict()

for filename in tqdm(os.listdir(user_reviews_dir)):
    if AMOUNT_TO_LOAD is not None and len(user_review_data) >= AMOUNT_TO_LOAD:
        break
    try:
        user_review_data[filename] = pd.read_csv(os.path.join(user_reviews_dir, filename), encoding='unicode_escape')
    except pd.errors.EmptyDataError:
        print(f'Empty file: {filename}')
        pass

  1%|          | 442/63111 [00:00<01:40, 621.48it/s]

Empty file: 468889434_reviews.csv


  4%|▍         | 2700/63111 [00:04<01:40, 601.15it/s]

Empty file: alinetta_reviews.csv


  9%|▊         | 5521/63111 [00:09<01:35, 603.86it/s]

...

 25%|██▌       | 15965/63111 [00:26<01:22, 570.30it/s]

Empty file: ds612_reviews.csv


 27%|██▋       | 16990/63111 [00:28<01:14, 617.41it/s]

Empty file: elinesophie_reviews.csv


 32%|███▏      | 20009/63111 [00:32<01:11, 606.66it/s]


Now let's split the data into training, validation, and test sets. Since this is a recommender, we're gonna split by removing some of the user's reviews.

For every user, so long as the user has more than 5 reviews, remove one review for the validation set and one review for the test set.

In [8]:
print(list(user_review_data.keys())[0])

0001kidd_reviews.csv


In [9]:
train_reviews = []
validation_reviews = []
test_reviews = []
for user_id, reviews in tqdm(user_review_data.items()):
    if len(reviews) > 5:
        # randomly remove one review from the user's reviews for the test set and one for the validation set
        reviews_to_remove = reviews.sample(2)
        # test data
        test_review_data = reviews_to_remove.iloc[0].to_dict()
        test_review_data['user_id'] = user_id
        test_reviews.append(test_review_data)
        # validation data
        validation_review_data = reviews_to_remove.iloc[1].to_dict()
        validation_review_data['user_id'] = user_id
        validation_reviews.append(validation_review_data)
        # train data
        train_review_data = reviews.drop(reviews_to_remove.index).to_dict('records')
        for review in train_review_data:
            review['user_id'] = user_id
        
        if (user_id == "ahmad97_reviews.csv"):
          print(len(train_review_data), len(reviews_to_remove))
        train_reviews.extend(train_review_data)
    else:
        # if the user has less than 5 reviews, we will use all of them for training
        train_review_data = reviews.to_dict('records')
        for review in train_review_data:
            review['user_id'] = user_id
        train_reviews.extend(train_review_data)

print(f'Train reviews: {len(train_reviews)}')
print(f'Validation reviews: {len(validation_reviews)}')
print(f'Test reviews: {len(test_reviews)}')

  8%|▊         | 1633/20000 [00:04<00:44, 414.43it/s]

518 2


100%|██████████| 20000/20000 [00:58<00:00, 339.72it/s]

Train reviews: 10529356
Validation reviews: 19711
Test reviews: 19711


In [10]:
train_reviews[0]

{'movie_title': 'M3GAN',
 'movie_rating': 3.0,
 'movie_id': 465649,
 'film_slug': '/film/m3gan/',
 'user_id': '0001kidd_reviews.csv'}

## Build the Model
Now that we have the training data, let's construct the model to train.

In [11]:
num_train_users = len(set([review['user_id'] for review in train_reviews]))
num_train_items = len(set([review['movie_id'] for review in train_reviews]))
num_nodes = num_train_users + num_train_items
print(f'Number of train users: {num_train_users}')
print(f'Number of train items: {num_train_items}')
print(f'Number of nodes: {num_nodes}')

Number of train users: 20000
Number of train items: 218499
Number of nodes: 238499


In [12]:
# Let's map users to ids
movie_id_to_movie_name = dict()
for review in train_reviews:
    movie_id_to_movie_name[review['movie_id']] = review['movie_title']

user_to_id = dict()
for i, user_id in enumerate(set([review['user_id'] for review in train_reviews])):
    user_to_id[user_id] = i

# Let's map movies to ids
movie_to_id = dict()
for i, movie_id in enumerate(set([review['movie_id'] for review in train_reviews])):
    movie_to_id[movie_id] = i + num_train_users

# Let's map ids to users
id_to_user = dict()
for user_id, index in user_to_id.items():
    id_to_user[index] = user_id

# Let's map ids to movies
id_to_movie = dict()
for movie_id, index in movie_to_id.items():
    id_to_movie[index] = movie_id

# Let's map movie names to movie ids
movie_name_to_movie_id = dict()
for movie_id, movie_name in movie_id_to_movie_name.items():
    movie_name_to_movie_id[movie_name] = movie_id

In [13]:
# Let's remove any data in our validation and test sets that have ids that are not in our training set
# Before removal:
print(f'Validation reviews: {len(validation_reviews)}')
print(f'Test reviews: {len(test_reviews)}')

# Removal
validation_reviews = [review for review in validation_reviews if review['user_id'] in user_to_id and review['movie_id'] in movie_to_id]
test_reviews = [review for review in test_reviews if review['user_id'] in user_to_id and review['movie_id'] in movie_to_id]

# After removal:
print(f'Validation reviews: {len(validation_reviews)}')
print(f'Test reviews: {len(test_reviews)}')

Validation reviews: 19711
Test reviews: 19711
Validation reviews: 19659
Test reviews: 19649


In [14]:
import random

def convert_review_to_edge(review):
    user_id = user_to_id[review['user_id']]
    movie_id = movie_to_id[review['movie_id']]
    edge_weight = review['movie_rating']
    if (edge_weight < 3.5 and edge_weight > 2.5):
        return None, None
    edge = (user_id, movie_id)
    edge_weight = review['movie_rating']
    return edge, edge_weight

def shuffle_edges_and_edge_weights(edges, edge_weights):
    c = list(zip(edges, edge_weights))
    random.shuffle(c)
    return zip(*c)

def convert_reviews_to_edges(reviews):
    edges = []
    edge_weights = []
    for review in tqdm(reviews):
        edge, edge_weight = convert_review_to_edge(review)
        if edge is not None:
            edges.append(edge)
            edge_weights.append(edge_weight)
    
    # Reformat the edges to be a tensor
    edges = torch.tensor(edges, dtype=torch.long).t().contiguous()
    return edges, edge_weights

In [15]:
# Now let's create the edges between users and movies.
# The id of the user will be the index of the user in the user_to_id dict
# The id of the movie will be the index of the movie in the movie_to_id dict + the number of users

train_edges, train_edge_weights = convert_reviews_to_edges(train_reviews)
validation_edges, validation_edge_weights = convert_reviews_to_edges(validation_reviews)

print(f'Train edges: {train_edges.shape[1]}')
print(f'Validation edges: {validation_edges.shape[1]}')

100%|██████████| 19659/19659 [00:00<00:00, 613789.16it/s]

Train edges: 8835474
Validation edges: 16824


In [16]:
import torch_geometric.data as data

# create the graph
train_graph = data.Data(
    edge_index=train_edges,
    edge_attr=torch.tensor(train_edge_weights),
    num_nodes=num_nodes
)

validation_graph = data.Data(
    edge_index=validation_edges,
    edge_attr=torch.tensor(validation_edge_weights),
    num_nodes=num_nodes
)

In [17]:
train_graph.validate(raise_on_error=True)
validation_graph.validate(raise_on_error=True)

True

In [18]:
# Let's create some negative edges
def resample_edges_for_user(user_positive_edges, user_negative_edges):
    num_negative_edges_to_add = user_positive_edges.shape[1] * 3 - user_negative_edges.shape[1]
    if (num_negative_edges_to_add <= 0):
        num_negative_edges_to_remove = -num_negative_edges_to_add
        # choose the negative edges to keep
        negative_edges_to_keep = torch.randint(user_negative_edges.shape[1], (user_negative_edges.shape[1] - num_negative_edges_to_remove,))
        # remove all the negative edges for this user
        user_negative_edges = user_negative_edges[:, negative_edges_to_keep]
    else:
        # Create new negative edges
        negative_edges_to_add = torch.tensor([[user_id] * num_negative_edges_to_add, torch.randint(num_train_users, num_train_items, (num_negative_edges_to_add,))], dtype=torch.long)
        # Add the negative edges to the negative edges for this user
        user_negative_edges = torch.cat([user_negative_edges, negative_edges_to_add], dim=1)
    return user_positive_edges, user_negative_edges
        

In [19]:
def compute_precision_at_k_memory_efficient(model, num_items, num_users, positive_edges, k=5):
    model.eval()
    # select a random subset of 1000 users
    users = torch.randperm(num_users)[:1000]
    # for each user, use a heapq to keep track of the top k items
    top_k_items = [list() for _ in range(1000)]
    print("Created top k items")
    with torch.no_grad():
        # we're going to go over all possible item, user pairs, but we're going to do it in batches
        for user_id in tqdm(users):
            all_edges_for_user = torch.tensor([(user_id, item_id) for item_id in range(num_items)], dtype=torch.long).t().contiguous()
            top_k_items[user_id] = model(all_edges_for_user).topk(k=k, dim=0)[1].tolist()
        # Check how many of the top k items are in the positive edges
        num_correct = 0
        for user_id, positive_items in enumerate(positive_edges):
            # increment num_correct if the edge (user_id, item_id) is in the positive edges
            num_correct += len(set(top_k_items[user_id]) & set(positive_items.tolist()))
        precision_at_k = num_correct / (num_users * k)
    return precision_at_k    

In [25]:
NUM_LAYERS = 1
LR = 0.00005
BATCH_SIZE = 256
model = LightGCN(num_nodes=num_nodes, embedding_dim=EMBEDDING_DIM, num_layers=NUM_LAYERS)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print("Running on device: {}".format(device))
print(EMBEDDING_DIM)

optim = torch.optim.Adam(model.parameters(), lr=LR)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optim, milestones=[100, 200, 300, 400], gamma=0.5)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim, T_0=100)

train_positive_edges = train_graph.edge_index[:, train_graph.edge_attr >= 3.5]
train_negative_edges = train_graph.edge_index[:, train_graph.edge_attr <= 2.5]


from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(comment=f'LightGCN_{EMBEDDING_DIM}_layers_{NUM_LAYERS}_batch_size_{BATCH_SIZE}_lr_{LR}_num_train_users_{num_train_users}_num_train_items_{num_train_items}')

for epoch in range(10001):
    # we are using BPR so we go by user
    average_loss = 0
    # We'll proceed in batches of users
    for start_idx in tqdm(range(0, num_train_users, BATCH_SIZE)):
        model.train()
        loss = torch.tensor(0.0, requires_grad=True)
        users_in_batch = torch.arange(start_idx, min(start_idx + BATCH_SIZE, num_train_users))
        for user_id in users_in_batch:
            # get all the edges specific to this user
            user_positive_edges = train_positive_edges[:, train_positive_edges[0] == user_id]
            user_negative_edges = train_negative_edges[:, train_negative_edges[0] == user_id]
            if (user_positive_edges.shape[1] == 0 or user_negative_edges.shape[1] == 0):
                continue
            # limit the number of positive edges to 5000
            if (user_positive_edges.shape[1] > 5000):
                user_positive_edges = user_positive_edges[:, :5000]
            # Get at most 15000 negative edges
            if (user_negative_edges.shape[1] > 15000):
                user_negative_edges = user_negative_edges[:, :15000]
            # resample the negative edges if we don't have enough
            user_positive_edges, user_negative_edges = resample_edges_for_user(user_positive_edges, user_negative_edges)
            # concatenate the positive and negative edges
            user_edges = torch.cat([user_positive_edges, user_negative_edges], dim=1)
            # get the rankings for this user
            user_edges = user_edges.to(device)
            user_rankings = model(user_edges)
            # divide the rankings into positive and negative rankings
            user_positive_rankings = user_rankings[:user_positive_edges.shape[1]]
            user_negative_rankings = user_rankings[user_positive_edges.shape[1]:]
            # create all pairs of positive and negative rankings
            user_positive_rankings = user_positive_rankings.unsqueeze(1).repeat(1, user_negative_rankings.shape[0])
            user_negative_rankings = user_negative_rankings.unsqueeze(0).repeat(user_positive_rankings.shape[0], 1)
            # get the user loss
            user_loss = model.recommendation_loss(user_positive_rankings, user_negative_rankings, 1e-4)
            # add the user loss to the total loss
            loss = loss + user_loss
        # divide the loss by the number of users
        loss = loss / BATCH_SIZE
        # log the loss
        # backprop
        optim.zero_grad()
        loss.backward()
        optim.step()
        writer.add_scalar("Loss/train", loss, epoch * (num_train_users // BATCH_SIZE) + start_idx // BATCH_SIZE)
        print(epoch * BATCH_SIZE + start_idx // BATCH_SIZE)
        average_loss = 0
        if (start_idx // BATCH_SIZE) % 50 == 0:
            model.eval()
            average_number_of_matches = 0
            for user_id in range(500):
                all_edges = torch.tensor([(user_id, item_id) for item_id in range(num_train_users, num_train_items)], dtype=torch.long).t().contiguous()
                recommendations = model.recommend(all_edges.to(device), src_index=torch.tensor([user_id]), dst_index=torch.tensor([x for x in range(num_train_users, num_train_items)]), k=10)[0]
                movie_names = [movie_id_to_movie_name[id_to_movie[int(recommendation)]] for recommendation in recommendations]
                true_user_reviews = user_review_data[id_to_user[user_id]]
                matches = 0
                for movie_name in movie_names:
                    if movie_name in true_user_reviews['movie_title'].values:
                        matches += 1
                average_number_of_matches += matches
            average_number_of_matches = average_number_of_matches / 500
            print("Average number of matches: {}".format(average_number_of_matches))
            writer.add_scalar("Average number of matches", average_number_of_matches, epoch * (num_train_users // BATCH_SIZE) + start_idx // BATCH_SIZE)
            print("=====================================")
    # scheduler.step()

    # # Compute the validation loss every 10 epochs
    # if (epoch % 10 == 0):
    #     model.eval()
    #     with torch.no_grad():
    #         val_loss = torch.tensor(0.0)
    #         for user_id in range(num_train_users):
    #             # get all the edges specific to this user
    #             user_positive_edges = train_positive_edges[:, train_positive_edges[0] == user_id]
    #             user_negative_edges = train_negative_edges[:, train_negative_edges[0] == user_id]
    #             if (user_positive_edges.shape[1] == 0 or user_negative_edges.shape[1] == 0):
    #                 continue
    #             # resample the negative edges if we don't have enough
    #             user_positive_edges, user_negative_edges = resample_edges_for_user(user_positive_edges, user_negative_edges)
    #             # concatenate the positive and negative edges
    #             user_edges = torch.cat([user_positive_edges, user_negative_edges], dim=1)
    #             # get the rankings for this user
    #             user_edges = user_edges.to(device)
    #             user_rankings = model(user_edges)
    #             # divide the rankings into positive and negative rankings
    #             user_positive_rankings = user_rankings[:user_positive_edges.shape[1]]
    #             user_negative_rankings = user_rankings[user_positive_edges.shape[1]:]
    #             # create all pairs of positive and negative rankings
    #             user_positive_rankings = user_positive_rankings.unsqueeze(1).repeat(1, user_negative_rankings.shape[0])
    #             user_negative_rankings = user_negative_rankings.unsqueeze(0).repeat(user_positive_rankings.shape[0], 1)
    #             # get the user loss
    #             user_loss = model.recommendation_loss(user_positive_rankings, user_negative_rankings, 1e-4)
    #             # add the user loss to the total loss
    #             val_loss = val_loss + user_loss
    #         # divide the loss by the number of users
    #         val_loss = val_loss / num_train_users
    #         # log the loss
    #         writer.add_scalar("Loss/val", val_loss, epoch)
    #         print("Epoch: {}, Loss: {}, Val Loss: {}".format(epoch, loss, val_loss))

    # Let's print the top ranked movies for a few users

Running on device: cuda
128


  0%|          | 0/79 [00:00<?, ?it/s]

0


  1%|▏         | 1/79 [00:34<44:57, 34.58s/it]

Average number of matches: 0.048


...

3887


 62%|██████▏   | 49/79 [03:20<01:49,  3.65s/it]

3888


 62%|██████▏   | 49/79 [03:22<02:04,  4.14s/it]


KeyboardInterrupt: 